# Confluence + LLM = QA

In [1]:
%pip install pandas --quiet
import pandas as pd

Note: you may need to restart the kernel to use updated packages.


In [12]:
from os import environ
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env")

confluence_token = environ.get('CONFLUENCE_TOKEN')
hf_token = environ.get('HF_TOKEN')
hf_write_token = environ.get('HF_WRITE_TOKEN')
gigachat_token = environ.get('GIGACHAT_TOKEN')

In [3]:
from atlassian import Confluence

confluence_url = "https://confluence.utmn.ru"
confluence = Confluence(url=confluence_url, token=confluence_token)

## Поиск документа в Confluence через CQL

 * https://atlassian-python-api.readthedocs.io/confluence.html
 * https://developer.atlassian.com/cloud/confluence/rest/v2/api-group-page/#api-pages-id-get
 * https://developer.atlassian.com/server/confluence/advanced-searching-using-cql/
 * https://spacy.io/usage/spacy-101

In [8]:
!python -m spacy download ru_core_news_sm --quiet

✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [4]:
from bs4 import BeautifulSoup
from langchain_community.document_loaders import PyPDFLoader

import spacy
nlp = spacy.load("ru_core_news_sm")
needed_pos = ['NOUN', 'NUM', 'PROPN', 'ADJ', 'VERB', 'X']


def get_cql_query(spaces, question):
    exclude = ' and label != "навигация"' 
    words = [token for token in nlp(question.lower()) if not token.is_stop and
             token.pos_ in needed_pos and len(token.text) > 2]
    if len(words) == 0:
        return ()
    spaces = " or ".join([f"space = {space}" for space in spaces])
    words_with_verbs = " and ".join(list(set([f"(title ~ '{word}*' or text ~ '{word}*' or title ~ '{word.lemma_}*' or text ~ '{word.lemma_}*')"
                                              for word in words])))
    words_without_verbs = " and ".join(list(set([f"(title ~ '{word}*' or text ~ '{word}*' or title ~ '{word.lemma_}*' or text ~ '{word.lemma_}*')"
                                                 for word in words if word.pos_ != 'VERB'])))
    words_without_verbs_and_adj = " and ".join(list(set([f"(title ~ '{word}*' or text ~ '{word}*' or title ~ '{word.lemma_}*' or text ~ '{word.lemma_}*')"
                                                         for word in words if word.pos_ not in ['VERB', 'ADJ']])))
    return ("(" + spaces + ") and (" + words_with_verbs + ")" + exclude,
            "(" + spaces + ") and (" + words_without_verbs + ")" + exclude,
            "(" + spaces + ") and (" + words_without_verbs_and_adj + ")" + exclude)
    
    
def get_document_id(question: str) -> str:
    cql_query = get_cql_query(spaces=["study"], question=question)
    if len(cql_query) == 0:
        return "0"
    results = confluence.cql(cql_query[0], start=0, limit=1)['results']
    if len(results) == 0:
        results = confluence.cql(cql_query[1], start=0, limit=1)['results']
        if len(results) == 0:
            results = confluence.cql(cql_query[2], start=0, limit=1)['results']
            if len(results) == 0:
                return "0"

    return results[0]['content']['id']


def get_document_content_by_id(page_id: str):
    page = confluence.get_page_by_id(page_id, expand='space,body.export_view')
    page_body = page['body']['export_view']['value']
    page_download = page['_links']['base'] + page['_links']['download'] if 'download' in page['_links'].keys() else ''

    try:
        if len(page_body) > 50:
            page_body = page['body']['export_view']['value']
            soup = BeautifulSoup(page_body, 'html.parser')
            page_body_text = soup.get_text(separator=' ')
            content = page_body_text.replace(" \n ", "")
        elif '.pdf' in page_download.lower():
            loader = PyPDFLoader(page_download.split('?')[0])
            content = " ".join([page.page_content for page in loader.load_and_split()])
        else:
            return None
    except:
        return None

    return content


def get_document_content(question: str):
    page_id = get_document_id(question)
    if page_id == "0":
        return None
    return get_document_content_by_id(page_id)

In [5]:
get_document_content("Как поменять физкультуру?")

'Выбор спортивных секций по Физической культуре будет проходить в ИС Модеус во вкладке "Выбор модулей".\xa0 Вам нужно будет выбрать 2 интересующие Вас спортивные секции, которые будут проходить каждую неделю в одно и то же время.\xa0 Ограничения: 1) Записаться можно не более чем на 2 занятия в неделю 2) Нельзя записываться на два занятия подряд.\xa0 Вас могут не допустить на занятие, если Вы были на предыдущей паре и/или уже посетили два занятия за неделю. При этом расписание на наличие конфликтов Вы проверяете самостоятельно в соответствии с Вашим расписанием в ИС Модеус и расписанием спортивных секций (во вложенных файлах). Ваш выбор пролонгируется до конца семестра, однако в любой момент Вы можете его изменить, отписавшись от одной секции и записавшись на другую. ВАЖНО!  Студент, пропустивший два занятия подряд, будет отписан автоматически. Выбор Физической культуры откроется 06.09.2023 и будет открыт до конца семестра.\xa0 Для успешной аттестации по дисциплине «Физическая культура:

## Собственный индекс Confluence

### Структура БД

In [13]:
from sqlalchemy import create_engine
engine = create_engine(f"postgresql://{environ.get('POSTGRES_USER')}:{environ.get('POSTGRES_PASSWORD')}@{environ.get('POSTGRES_HOST')}/{environ.get('POSTGRES_DB')}", echo=False)

In [14]:
from typing import Optional
from pgvector.sqlalchemy import Vector
from sqlalchemy import Text
from sqlalchemy.orm import DeclarativeBase, Mapped, Session, mapped_column

class Base(DeclarativeBase):
    pass

class Chunk(Base):
    __tablename__ = "chunk"
    id: Mapped[int] = mapped_column(primary_key=True)
    confluence_id: Mapped[int] = mapped_column(index=True)
    text: Mapped[str] = mapped_column(Text())
    text_lem: Mapped[str] = mapped_column(Text())
    tfidf: Mapped[Optional[Vector]] = mapped_column(Vector(2777))
    doc2vec: Mapped[Optional[Vector]] = mapped_column(Vector(150))
    rubert: Mapped[Optional[Vector]] = mapped_column(Vector(312))
    rusbert: Mapped[Optional[Vector]] = mapped_column(Vector(312))
    rusbert_finetuned: Mapped[Optional[Vector]] = mapped_column(Vector(312))
    gigachat_embeddings: Mapped[Optional[Vector]] = mapped_column(Vector(1024))

In [8]:
def lower_stopword_lemmatize(text):
    return " ".join([token.lemma_ for token in nlp(str(text).lower()) if not token.is_stop and token.pos_ != 'PUNCT'])

### Выгрузка, предобработка и сохранение документов из пространства

In [9]:
page_ids = []
count_start = 0
limit = 100
pages = confluence.cql(f"space = study and label != \"навигация\" order by id", start=count_start, limit=limit)["results"]
while len(pages) != 0:
    page_ids = page_ids + [page['content']['id'] for page in pages if 'content' in page.keys()]
    count_start += limit
    pages = confluence.cql(f"space = study and label != \"навигация\" order by id", start=count_start, limit=limit)["results"]
page_ids

['86478987',
 '86478990',
 '86478992',
 '86478994',
 '86478996',
 '86479005',
 '86479007',
 '86479009',
 '86479011',
 '86479013',
 '86479015',
 '86479017',
 '86479021',
 '86479024',
 '86479026',
 '86479028',
 '86479030',
 '86479032',
 '86479034',
 '86479036',
 '86479041',
 '86479043',
 '86479045',
 '86479047',
 '86479050',
 '86479052',
 '86479054',
 '86479056',
 '86479060',
 '86479065',
 '86479075',
 '86479076',
 '86479078',
 '86479079',
 '86479083',
 '86479085',
 '86479086']

In [14]:
from langchain.text_splitter import SentenceTransformersTokenTextSplitter
from langchain_core.documents import Document

documents = []
for page_id in page_ids:
    page_content = get_document_content_by_id(page_id)
    if page_content is None:
        continue
    documents.append(Document(
        page_content=page_content, metadata={"page_id": page_id}
    ))
documents[:3]

[Document(page_content='Сформировать справку-вызов Вы можете самостоятельно в личном кабинете на портале "Вместе"  https://vmeste.utmn.ru/login  (раздел "Обучение"=>"Справки"). После присвоения Вашей заявке статуса "Выполнено" на почту  ed@utmn.ru  необходимо сообщить о способе, которым Вы планируете забрать готовую справку. ВАЖНО! На основании ст. 173 Трудового кодекса РФ и п. 4.5. Положения о текущем контроле успеваемости и промежуточной аттестации обучающихся ФГАОУ ВО ТюмГУ справка-вызов на текущую сессию студентам заочной формы обучения оформляется только при успешной сдаче предыдущей сессии и предоставляется до начала сессии. \xa0 При наличии академической задолженности за предыдущую сессию, справка-вызов не предоставляется.', metadata={'page_id': '86478987'}),
 Document(page_content='Вам необходимо обратиться в Отдел мобилизационной подготовки по адресу: ул. Перекопская, 15А каб.408\xa0 Телефон: (3452) 59-75-83', metadata={'page_id': '86478990'}),
 Document(page_content='Студенты

In [15]:
text_splitter = SentenceTransformersTokenTextSplitter(model_name="saved_models/rubert-tiny2-wikiutmn")   
all_splits = text_splitter.split_documents(documents)
all_splits

[Document(page_content='Сформировать справку - вызов Вы можете самостоятельно в личном кабинете на портале " Вместе " https : / / vmeste. utmn. ru / login ( раздел " Обучение " = > " Справки " ). После присвоения Вашей заявке статуса " Выполнено " на почту ed @ utmn. ru необходимо сообщить о способе, которым Вы планируете забрать готовую справку. ВАЖНО! На основании ст. 173 Трудового кодекса РФ и п. 4. 5. Положения о текущем контроле успеваемости и промежуточной аттестации обучающихся ФГАОУ ВО ТюмГУ справка - вызов на текущую сессию студентам заочной формы обучения оформляется только при успешной сдаче предыдущей сессии и предоставляется до начала сессии. При наличии академической задолженности за предыдущую сессию, справка - вызов не предоставляется.', metadata={'page_id': '86478987'}),
 Document(page_content='Вам необходимо обратиться в Отдел мобилизационной подготовки по адресу : ул. Перекопская, 15А каб. 408 Телефон : ( 3452 ) 59 - 75 - 83', metadata={'page_id': '86478990'}),
 Docu

In [21]:
from sqlalchemy import text

with Session(engine) as session:
    session.execute(text('CREATE EXTENSION IF NOT EXISTS vector'))
    session.commit()
Base.metadata.create_all(engine)

In [22]:
with Session(engine) as session:
    for split in all_splits:
        doc = Chunk(
            confluence_id=int(split.metadata["page_id"]),
            text=split.page_content, 
            text_lem=lower_stopword_lemmatize(split.page_content)
        )
        session.add(doc)
    session.commit()   

### Эмбеддинги

In [9]:
from sqlalchemy import select

with Session(engine) as session:
   db_documents_lem = pd.Series([doc.text_lem for doc in session.scalars(select(Chunk).order_by(Chunk.id)).all()])
db_documents_lem.head()

0    сформировать справка - вызов самостоятельно ли...
1    обратиться отдел мобилизационный подготовка ад...
2    студент очный форма обучение оформлять справка...
3    оформление академический справка справка перио...
4    вопрос получение справка доход размер стипенди...
dtype: object

#### TFIDF

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidfX = tfidf_vectorizer.fit_transform(db_documents_lem)
len(tfidf_vectorizer.transform([db_documents_lem[0]]).toarray()[0])

2777

In [11]:
with Session(engine) as session:
   documents = session.scalars(select(Chunk).order_by(Chunk.id)).all()
   for doc in documents:
      doc.tfidf = tfidf_vectorizer.transform([doc.text_lem]).toarray()[0]
      session.add(doc)
      session.flush()
   session.commit()

In [12]:
def answer_tfidf(question):
    with Session(engine) as session:
        return session.scalars(select(Chunk)
                        .order_by(Chunk.tfidf.cosine_distance(
                            tfidf_vectorizer.transform([lower_stopword_lemmatize(question)]).toarray()[0]
                            )).limit(1)).first().text

In [13]:
answer_tfidf("Как поменять физкультуру?")

'Прежде, чем выбрать элективы, рекомендуем почитать отзывы на сервисе « Отзывус » : https : / / electives. utmn. ru. Там же можно написать о своём опыте изучения элективных дисциплин. Информация, о том, как поменять электив, отправляется на корпоративную почту в первую учебную неделю семестра.'

#### Doc2vec

In [29]:
%pip install gensim --quiet

Note: you may need to restart the kernel to use updated packages.


In [14]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate([text.split() for text in db_documents_lem])]
doc2vec_model = Doc2Vec(documents, vector_size=150, window=5, min_count=1, workers=4)

len(doc2vec_model.infer_vector("мама мыла раму".split()))

150

In [29]:
with Session(engine) as session:
   documents = session.scalars(select(Chunk).order_by(Chunk.id)).all()
   for doc in documents:
      doc.doc2vec = doc2vec_model.infer_vector(doc.text_lem.split())
      session.add(doc)
      session.flush()
   session.commit()

In [15]:
def answer_doc2vec(question):
    with Session(engine) as session:
        return session.scalars(select(Chunk)
                        .order_by(Chunk.doc2vec.cosine_distance(
                           doc2vec_model.infer_vector(lower_stopword_lemmatize(question).split())
                            )).limit(1)).first().text

In [16]:
answer_doc2vec("Как поменять физкультуру?")

'Заявление подается через личный кабинет на портале " Вместе ". Отпуск по уходу за ребенком до достижения им возраста трех лет предоставляется обучающемуся, являющемуся матерью ( отцом, опекуном ) на основании личного заявления, к которому прилагается копия свидетельства о рождении ребенка. Отпуск по уходу за ребенком до достижения им возраста трех лет оформляется сроком не более трех лет с момента рождения ребенка и может быть использован полностью или по частям.'

#### RuBERT-Tiny

In [17]:
import torch
from transformers import AutoTokenizer, AutoModel
rubert_tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
rubert_model = AutoModel.from_pretrained("cointegrated/rubert-tiny")
rubert_model.cpu()
# rubert_model.cuda()  # uncomment it if you have a GPU

def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

len(embed_bert_cls("мама мыла раму", rubert_model, rubert_tokenizer))

312

In [33]:
with Session(engine) as session:
   documents = session.scalars(select(Chunk).order_by(Chunk.id)).all()
   for doc in documents:
      doc.rubert = embed_bert_cls(doc.text, rubert_model, rubert_tokenizer)
      session.add(doc)
      session.flush()
   session.commit()

In [18]:
def answer_rubert(question):
    with Session(engine) as session:
        return session.scalars(select(Chunk)
                        .order_by(Chunk.rubert.cosine_distance(
                            embed_bert_cls(question, rubert_model, rubert_tokenizer)
                            )).limit(1)).first().text

In [19]:
answer_rubert("Как поменять физкультуру?")

'Прежде, чем выбрать элективы, рекомендуем почитать отзывы на сервисе « Отзывус » : https : / / electives. utmn. ru. Там же можно написать о своём опыте изучения элективных дисциплин. Информация, о том, как поменять электив, отправляется на корпоративную почту в первую учебную неделю семестра.'

#### RuSBERT-Tiny

In [20]:
from sentence_transformers import SentenceTransformer

rusbert_model = SentenceTransformer('cointegrated/rubert-tiny2', device="cpu")
len(rusbert_model.encode("мама мыла раму"))

312

In [37]:
with Session(engine) as session:
   documents = session.scalars(select(Chunk).order_by(Chunk.id)).all()
   for doc in documents:
      doc.rusbert = rusbert_model.encode(doc.text)
      session.add(doc)
      session.flush()
   session.commit()

In [21]:
def answer_rusbert(question):
    with Session(engine) as session:
        return session.scalars(select(Chunk)
                        .order_by(Chunk.rusbert.cosine_distance(
                            rusbert_model.encode(question)
                            )).limit(1)).first().text

In [22]:
answer_rusbert("Как поменять физкультуру?")

'Выбор спортивных секций по Физической культуре будет проходить в ИС Модеус во вкладке " Выбор модулей ". Вам нужно будет выбрать 2 интересующие Вас спортивные секции, которые будут проходить каждую неделю в одно и то же время. Ограничения : 1 ) Записаться можно не более чем на 2 занятия в неделю 2 ) Нельзя записываться на два занятия подряд. Вас могут не допустить на занятие, если Вы были на предыдущей паре и / или уже посетили два занятия за неделю. При этом расписание на наличие конфликтов Вы проверяете самостоятельно в соответствии с Вашим расписанием в ИС Модеус и расписанием спортивных секций ( во вложенных файлах ). Ваш выбор пролонгируется до конца семестра, однако в любой момент Вы можете его изменить, отписавшись от одной секции и записавшись на другую. ВАЖНО! Студент, пропустивший два занятия подряд, будет отписан автоматически. Выбор Физической культуры откроется 06. 09. 2023 и будет открыт до конца семестра. Для успешной аттестации по дисциплине « Физическая культура : эле

#### GigaChatEmbeddings

In [7]:
%pip install --upgrade --quiet  gigachain

Note: you may need to restart the kernel to use updated packages.


In [23]:
from langchain_community.embeddings import GigaChatEmbeddings

gigachat_embeddings = GigaChatEmbeddings(credentials=gigachat_token, verify_ssl_certs=False)
result = gigachat_embeddings.embed_documents(texts=["Привет!"])
print(len(result[0]))

1024


In [25]:
with Session(engine) as session:
   documents = session.scalars(select(Chunk).order_by(Chunk.id)).all()
   for doc in documents:
      try:
         doc.gigachat_embeddings = gigachat_embeddings.embed_documents(texts=[doc.text])[0]
      except:
         print(doc.text)
         doc.gigachat_embeddings = None
      session.add(doc)
      session.flush()
   session.commit()

Документ подписан простой электронной подписью Информация о владельце : ФИО : Романчук Иван Сергеевич Должность : Ректор Дата подписания : 13. 06. 2023 17 : 47 : 41 Уникальный программный ключ : 6319edc2b582ffdacea443f01d5779368d0957ac34f5cd074d81181530452479 2 1. ОБЩИЕ ПОЛОЖЕНИЯ 1. 1. Положение о порядке отчисления, восстановления обучающихся в федерально м государственно м автономно м образовательно м учреждени и высшего образования « Тюменский государственный университет » ( далее [UNK] Положение ) разработано в соответствии с : [UNK] Федеральным законом от 29. 12. 2012 № 273 - ФЗ « Об образовании в Российской Федерации » ; [UNK] приказом Минпросвещения России от 24. 08. 2022 № 762 « Об утверждении Порядка организации и осуществления образовательной деятельности по образовательным программам среднего профессионального образования » ; [UNK] приказом Министерства науки и высшего образования Российской Федерации от 06. 04. 2021 № 245 « Об утверждении Порядка организации и осуществления

In [26]:
def answer_gigachat_embeddings(question):
    with Session(engine) as session:
        return session.scalars(select(Chunk)
                        .order_by(Chunk.gigachat_embeddings.cosine_distance(
                            gigachat_embeddings.embed_documents(texts=[question])[0]
                            )).limit(1)).first().text

In [29]:
answer_gigachat_embeddings("Как поменять физкультуру?")

'Прежде, чем выбрать элективы, рекомендуем почитать отзывы на сервисе « Отзывус » : https : / / electives. utmn. ru. Там же можно написать о своём опыте изучения элективных дисциплин. Информация, о том, как поменять электив, отправляется на корпоративную почту в первую учебную неделю семестра.'

## Выбор нужного фрагмента через векторный индекс

### Тестовая выборка с вопросами

In [2]:
study_questions = pd.read_csv("study_questions.csv", index_col=0)
study_questions

,question,page_id,document
0,Почему не могу получить справку-вызов?,86478987,Сформировать справку - вызов Вы можете самосто...
1,Как получить справку-вызов?,86478987,Сформировать справку - вызов Вы можете самосто...
2,Где получить справку-вызов?,86478987,Сформировать справку - вызов Вы можете самосто...
3,Где взять справку для военкомата?,86478990,Вам необходимо обратиться в Отдел мобилизацион...
4,Как получить отсрочку от армии?,86478990,Вам необходимо обратиться в Отдел мобилизацион...
...,...,...,...
106,Могу ли я в последний момент отказаться от сме...,86479065,"Прежде, чем выбрать элективы, рекомендуем почи..."
107,где почитать отзывы на элективы?,86479065,"Прежде, чем выбрать элективы, рекомендуем почи..."
108,когда можно поменять элективы?,86479065,"Прежде, чем выбрать элективы, рекомендуем почи..."
109,как выбрать электив?,86479065,"Прежде, чем выбрать элективы, рекомендуем почи..."


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(study_questions.question, study_questions.document, test_size=0.6, random_state=666, stratify=study_questions.document)
test_questions = pd.DataFrame({"question": X_test, "document": y_test}).reset_index(drop=True)
test_questions.head()

,question,document
0,Я потерял магнитную карту. К кому обратиться?,Для восстановления студенческого билета Вам не...
1,Даты подачи заявления для восстановления на очку,Заявления на восстановление в Университет по о...
2,Что делать при потере проходки?,Для восстановления студенческого билета Вам не...
3,Я хожу в фитнес-клуб. Как заменить физкультуру?,Выбор спортивных секций по Физической культуре...
4,Когда мне выдадут студенческий билет после пер...,"1. Подать заявление о переводе можно лично, об..."


In [4]:
train_questions = pd.DataFrame({"question": X_train, "document": y_train}).reset_index(drop=True)
train_questions.head()

,question,document
0,Где продлять студак?,Для восстановления студенческого билета Вам не...
1,Когда можно получить справку о стипендии?,"По вопросу получения справки о доходах, размер..."
2,Как перевестись на другое направление на заочке?,Заявления о переводе принимаются два раза в го...
3,Как перевестись на другое направление?,Заявления о переводе принимаются два раза в го...
4,Могу ли я в последний момент отказаться от сме...,"Прежде, чем выбрать элективы, рекомендуем почи..."


In [33]:
test_questions["CQL"] = test_questions["question"].apply(get_document_content)
test_questions["tfidf"] = test_questions["question"].apply(answer_tfidf)
test_questions["doc2vec"] = test_questions["question"].apply(answer_doc2vec)
test_questions["rubert"] = test_questions["question"].apply(answer_rubert)
test_questions["rusbert"] = test_questions["question"].apply(answer_rusbert)
test_questions["gigachatembeddings"] = test_questions["question"].apply(answer_gigachat_embeddings)
test_questions

,question,document,CQL,tfidf,doc2vec,rubert,rusbert,gigachatembeddings
0,Я потерял магнитную карту. К кому обратиться?,Для восстановления студенческого билета Вам не...,Для восстановления студенческого билета Вам ...,Для восстановления студенческого билета Вам не...,Заявление подается через личный кабинет на пор...,Для восстановления студенческого билета Вам не...,Для восстановления студенческого билета Вам не...,Для восстановления студенческого билета Вам не...
1,Даты подачи заявления для восстановления на очку,Заявления на восстановление в Университет по о...,None,При равном числе голосов председатель аттестац...,"1. Подать заявление о переводе можно лично, об...",Заявление подается через личный кабинет на пор...,"При восстановлении на договорное место, после ...",Заявления на восстановление в Университет по о...
2,Что делать при потере проходки?,Для восстановления студенческого билета Вам не...,Для восстановления студенческого билета Вам ...,Для восстановления студенческого билета Вам не...,Заявление подается через личный кабинет на пор...,Для рассмотрения возможности продления академи...,Вам необходимо обратиться в Отдел мобилизацион...,Для восстановления студенческого билета Вам не...
3,Я хожу в фитнес-клуб. Как заменить физкультуру?,Выбор спортивных секций по Физической культуре...,None,Выбор спортивных секций по Физической культуре...,Заявление подается через личный кабинет на пор...,"Прежде, чем выбрать элективы, рекомендуем почи...",Выбор спортивных секций по Физической культуре...,"Прежде, чем выбрать элективы, рекомендуем почи..."
4,Когда мне выдадут студенческий билет после пер...,"1. Подать заявление о переводе можно лично, об...",Документ подписан простой электронной подписью...,Подать заявление на отчисление переводом можно...,Заявление подается через личный кабинет на пор...,Заявление на выход из отпуска подается не позд...,"При восстановлении на договорное место, после ...",Подать заявление на отчисление переводом можно...
...,...,...,...,...,...,...,...,...
62,как закрыть физкультуру?,Выбор спортивных секций по Физической культуре...,Выбор спортивных секций по Физической культуре...,Выбор спортивных секций по Физической культуре...,Заявление подается через личный кабинет на пор...,Пакет документов для восстановления вам необхо...,Вам необходимо обратиться в Отдел мобилизацион...,Вам необходимо обратиться в Отдел мобилизацион...
63,когда можно поменять элективы?,"Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...",Заявление подается через личный кабинет на пор...,"Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи..."
64,как получить справку о месте учёбы?,Студенты очной формы обучения оформляют справк...,None,Для оформления академической справки ( справки...,_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ ( ФИО указать ...,Для рассмотрения возможности продления академи...,Подать заявление на отчисление переводом можно...,Заявление подается через личный кабинет на пор...
65,Как можно получать баллы за физру?,Выбор спортивных секций по Физической культуре...,None,Об утверждении Регламента проведения промежуто...,Об утверждении Регламента проведения промежуто...,Для рассмотрения возможности продления академи...,Сформировать справку - вызов Вы можете самосто...,Студенты очной формы обучения оформляют справк...


### Выбор лучшего алгоритма

#### Accuracy

In [40]:
from langchain.text_splitter import SentenceTransformersTokenTextSplitter

text_splitter = SentenceTransformersTokenTextSplitter(model_name="saved_models/rubert-tiny2-wikiutmn")   
test_questions["CQL"] = test_questions["CQL"].apply(lambda x: text_splitter.split_text(x)[0] if x is not None else None)
test_questions

,question,document,CQL,tfidf,doc2vec,rubert,rusbert,gigachatembeddings
0,Я потерял магнитную карту. К кому обратиться?,Для восстановления студенческого билета Вам не...,Для восстановления студенческого билета Вам не...,Для восстановления студенческого билета Вам не...,Заявление подается через личный кабинет на пор...,Для восстановления студенческого билета Вам не...,Для восстановления студенческого билета Вам не...,Для восстановления студенческого билета Вам не...
1,Даты подачи заявления для восстановления на очку,Заявления на восстановление в Университет по о...,None,При равном числе голосов председатель аттестац...,"1. Подать заявление о переводе можно лично, об...",Заявление подается через личный кабинет на пор...,"При восстановлении на договорное место, после ...",Заявления на восстановление в Университет по о...
2,Что делать при потере проходки?,Для восстановления студенческого билета Вам не...,Для восстановления студенческого билета Вам не...,Для восстановления студенческого билета Вам не...,Заявление подается через личный кабинет на пор...,Для рассмотрения возможности продления академи...,Вам необходимо обратиться в Отдел мобилизацион...,Для восстановления студенческого билета Вам не...
3,Я хожу в фитнес-клуб. Как заменить физкультуру?,Выбор спортивных секций по Физической культуре...,None,Выбор спортивных секций по Физической культуре...,Заявление подается через личный кабинет на пор...,"Прежде, чем выбрать элективы, рекомендуем почи...",Выбор спортивных секций по Физической культуре...,"Прежде, чем выбрать элективы, рекомендуем почи..."
4,Когда мне выдадут студенческий билет после пер...,"1. Подать заявление о переводе можно лично, об...",Документ подписан простой электронной подписью...,Подать заявление на отчисление переводом можно...,Заявление подается через личный кабинет на пор...,Заявление на выход из отпуска подается не позд...,"При восстановлении на договорное место, после ...",Подать заявление на отчисление переводом можно...
...,...,...,...,...,...,...,...,...
62,как закрыть физкультуру?,Выбор спортивных секций по Физической культуре...,Выбор спортивных секций по Физической культуре...,Выбор спортивных секций по Физической культуре...,Заявление подается через личный кабинет на пор...,Пакет документов для восстановления вам необхо...,Вам необходимо обратиться в Отдел мобилизацион...,Вам необходимо обратиться в Отдел мобилизацион...
63,когда можно поменять элективы?,"Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...",Заявление подается через личный кабинет на пор...,"Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи..."
64,как получить справку о месте учёбы?,Студенты очной формы обучения оформляют справк...,None,Для оформления академической справки ( справки...,_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ ( ФИО указать ...,Для рассмотрения возможности продления академи...,Подать заявление на отчисление переводом можно...,Заявление подается через личный кабинет на пор...
65,Как можно получать баллы за физру?,Выбор спортивных секций по Физической культуре...,None,Об утверждении Регламента проведения промежуто...,Об утверждении Регламента проведения промежуто...,Для рассмотрения возможности продления академи...,Сформировать справку - вызов Вы можете самосто...,Студенты очной формы обучения оформляют справк...


In [41]:
for column in test_questions.columns[1:]:
    print(column, sum(test_questions[column].apply(lambda x: "" if x is None else x) == test_questions.document) / len(test_questions.document))

document 1.0
CQL 0.417910447761194
tfidf 0.44776119402985076
doc2vec 0.05970149253731343
rubert 0.22388059701492538
rusbert 0.373134328358209
gigachatembeddings 0.5373134328358209


#### ROUGE-L

In [34]:
%pip install rouge --quiet

Note: you may need to restart the kernel to use updated packages.


In [42]:
from rouge import Rouge
rouge = Rouge()

for column in test_questions.columns[1:]:
    print(column, rouge.get_scores(test_questions[column].apply(lambda x: "-" if x is None else x), test_questions["document"], avg=True)['rouge-l'])

document {'r': 1.0, 'p': 1.0, 'f': 0.9999999950000011}
CQL {'r': 0.5068911712305725, 'p': 0.7031467924154658, 'f': 0.4662775835703285}
tfidf {'r': 0.6250521000134985, 'p': 0.5366987292663881, 'f': 0.5409536072305204}
doc2vec {'r': 0.18644225174242757, 'p': 0.18525011213055334, 'f': 0.15950926244958688}
rubert {'r': 0.3640849525108445, 'p': 0.40906272000096044, 'f': 0.36020195485400025}
rusbert {'r': 0.49024178922048917, 'p': 0.5069492824343246, 'f': 0.4804577409306676}
gigachatembeddings {'r': 0.6138765422480673, 'p': 0.6498791978849756, 'f': 0.6188192588396773}


## SBERT Fine Tuning

 * https://www.sbert.net/docs/training/overview.html
 * https://huggingface.co/blog/how-to-train-sentence-transformers

In [5]:
from sentence_transformers import SentenceTransformer

### Генерация обучающей выборки через GigaChat

In [44]:
with Session(engine) as session:
   db_documents = [doc.text for doc in session.scalars(select(Chunk).order_by(Chunk.id)).all()]
len(db_documents)

52

In [45]:
from langchain.prompts import PromptTemplate

prompt_template = """
Сделай глубокий вдох и действуй как студент. Составь 7 разнообразных вопросов к документу в тройных кавычках. Используй разговорный стиль речи и студенческую лексику.

\"\"\"
{content}
\"\"\"

Вопросы:
"""

prompt = PromptTemplate.from_template(prompt_template)

In [46]:
from langchain.llms import GigaChat
giga = GigaChat(credentials=gigachat_token, verify_ssl_certs=False)
giga_chain = prompt | giga
giga_chain.invoke({"content": db_documents[0]}).strip().split("\n")

['1. Что такое справка-вызов?',
 '  ',
 '2. Как можно сформировать справку-вызов?',
 '  ',
 '3. Какую информацию нужно предоставить после присвоения заявке статуса "Выполнено"?',
 '  ',
 '4. Какие требования предъявляются к студентам для получения справки-вызова?',
 '  ',
 '5. Какие документы регулируют процесс оформления справки-вызова?',
 '  ',
 '6. Каковы последствия отсутствия справки-вызова для студентов с академической задолженностью?',
 '  ',
 '7. Каким образом можно связаться с университетом для получения дополнительной информации о справке-вызове?']

In [47]:
gigachat_docs = []
for doc in db_documents:
    query = {"content": doc}
    giga_questions = giga_chain.invoke(query).strip().split("\n")
    for q in giga_questions:
        q = q.strip()
        if len(q) < 5:
            continue
        gigachat_docs.append({
            "question": q[3:],
            "document": doc
        })
    print(giga_questions)

['1. Что такое справка-вызов?', '  ', '2. Как можно сформировать справку-вызов?', '  ', '3. Какую информацию нужно предоставить после присвоения заявке статуса "Выполнено"?', '  ', '4. Какие требования предъявляются к студентам для получения справки-вызова?', '  ', '5. Какие документы регулируют процесс оформления справки-вызова?', '  ', '6. Каковы последствия отсутствия справки-вызова для студентов с академической задолженностью?', '  ', '7. Каким образом можно связаться с университетом для получения дополнительной информации о справке-вызове?']
['1. "Какие документы мне нужны для обращения в Отдел мобилизационной подготовки?"', '  ', '  2. "Какие услуги предоставляет Отдел мобилизационной подготовки?"', '   ', '  3. "Какие требования предъявляются к документам, которые я должен предоставить?"', '   ', '  4. "Какие сроки рассмотрения документов в Отделе мобилизационной подготовки?"', '   ', '  5. "Какие контактные данные у Отдела мобилизационной подготовки?"', '   ', '  6. "Какие проц

Giga generation stopped with reason: blacklist


['Что-то в вашем вопросе меня смущает. Может, поговорим на другую тему?']


Giga generation stopped with reason: blacklist


['Как у нейросетевой языковой модели у меня не может быть настроения, но почему-то я совсем не хочу говорить на эту тему.']
['1. Что требуется заполнить, если обучающийся планирует перевод на место, финансируемое за счет бюджетных ассигнований федерального бюджета?', '  ', '2. Что требуется заполнить, если лицо, претендующее на перевод, ранее обучалось на месте, финансируемом за счет бюджетных ассигнований федерального бюджета?', '', '3. Что требуется заполнить, если обучающийся планирует перевод на место, финансируемое за счет бюджетных ассигнований федерального бюджета?', '', '4. Что требуется заполнить, если обучающийся планирует перевод на место, финансируемое за счет бюджетных ассигнований федерального бюджета, и общая продолжительность обучения будет превышать более чем на один учебный год срок освоения образовательной программы?', '', '5. Что требуется заполнить, если обучающийся планирует перевод на место, финансируемое за счет бюджетных ассигнований федерального бюджета, и ран

...

['Как у нейросетевой языковой модели у меня не может быть настроения, но почему-то я совсем не хочу говорить на эту тему.']
['1. Каковы формы текущего контроля?', '2. Как преподаватель информирует обучающихся о текущем контроле?', '3. Какие инструменты и технологии используются для текущего контроля?', '4. Как осуществляется текущий контроль по практике?', '5. Как формируются промежуточные и окончательные оценки по дисциплине?', '6. Как осуществляется промежуточная аттестация?', '7. Как проводятся экзамены и зачеты по факультативным дисциплинам?']
['1. Каким образом проводится замена экзаменатора?', ' ', '2. Как формируются ведомости при проведении экзамена?', ' ', '3. Какова продолжительность письменного экзамена?', ' ', '4. Как должны быть оформлены письменные и устные ответы на вопросы?', ' ', '5. Какие права имеет обучающийся во время экзамена?', ' ', '6. Как объявляются результаты экзамена?', ' ', '7. Какие критерии используются при выставлении оценок за экзамен?']


Giga generation stopped with reason: blacklist


['Как у нейросетевой языковой модели у меня не может быть настроения, но почему-то я совсем не хочу говорить на эту тему.']
['1. Что такое промежуточная аттестация?', '2. Какие формы промежуточной аттестации существуют?', '3. Как проводится первая повторная промежуточная аттестация?', '4. Как проводится вторая повторная промежуточная аттестация?', '5. Как оцениваются результаты повторной промежуточной аттестации?', '6. Как оформляются результаты промежуточной аттестации?', '7. Кто несет ответственность за правильность оформления документов о промежуточной аттестации?']
['1. Что происходит с оценками после прохождения промежуточной аттестации?', '2. Кто несет ответственность за правильное внесение информации о результатах промежуточной аттестации?', '3. Кто отвечает за внесение информации о результатах ликвидации академической разницы?', '4. Как вносятся изменения и дополнения к Положению?', '5. Какие нормативные акты регулируют изменения и дополнения к Положению?', '6. Какие документы 

Giga generation stopped with reason: blacklist


['Как у нейросетевой языковой модели у меня не может быть настроения, но почему-то я совсем не хочу говорить на эту тему.']


In [48]:
gigachat_docs = pd.DataFrame(gigachat_docs)
gigachat_docs.to_csv("gigachat_docs.csv")

### Тонкая настройка

In [6]:
gigachat_docs = pd.read_csv("gigachat_docs.csv", index_col=0).reset_index(drop=True)
gigachat_docs

,question,document
0,Что такое справка-вызов?,Сформировать справку - вызов Вы можете самосто...
1,Как можно сформировать справку-вызов?,Сформировать справку - вызов Вы можете самосто...
2,Какую информацию нужно предоставить после прис...,Сформировать справку - вызов Вы можете самосто...
3,Какие требования предъявляются к студентам для...,Сформировать справку - вызов Вы можете самосто...
4,Какие документы регулируют процесс оформления ...,Сформировать справку - вызов Вы можете самосто...
...,...,...
306,Как оформляются результаты промежуточной аттес...,"проведения промежуточной аттестации, за исключ..."
307,Кто несет ответственность за правильность офор...,"проведения промежуточной аттестации, за исключ..."
308,Что происходит с оценками после прохождения пр...,". 6. 10. Из ведомости, за исключением электрон..."
309,Кто несет ответственность за правильное внесен...,". 6. 10. Из ведомости, за исключением электрон..."


In [7]:
train_questions = pd.concat([train_questions, gigachat_docs], ignore_index=True)
train_questions

,question,document
0,Где продлять студак?,Для восстановления студенческого билета Вам не...
1,Когда можно получить справку о стипендии?,"По вопросу получения справки о доходах, размер..."
2,Как перевестись на другое направление на заочке?,Заявления о переводе принимаются два раза в го...
3,Как перевестись на другое направление?,Заявления о переводе принимаются два раза в го...
4,Могу ли я в последний момент отказаться от сме...,"Прежде, чем выбрать элективы, рекомендуем почи..."
...,...,...
350,Как оформляются результаты промежуточной аттес...,"проведения промежуточной аттестации, за исключ..."
351,Кто несет ответственность за правильность офор...,"проведения промежуточной аттестации, за исключ..."
352,Что происходит с оценками после прохождения пр...,". 6. 10. Из ведомости, за исключением электрон..."
353,Кто несет ответственность за правильное внесен...,". 6. 10. Из ведомости, за исключением электрон..."


In [8]:
import math
from sentence_transformers import InputExample, losses
from torch.utils.data import DataLoader

train_set = []
for index, row in train_questions.iterrows():
    train_set.append(InputExample(texts=[row['question'], row['document']]))
    
finetuned_model = SentenceTransformer("cointegrated/rubert-tiny2", device="cuda")

train_dataloader = DataLoader(train_set, shuffle=True, batch_size=8)
train_loss = losses.MultipleNegativesRankingLoss(finetuned_model)
# train_loss = losses.MegaBatchMarginLoss(finetuned_model)

num_epochs = 10
warmup_steps = math.ceil(len(train_set) * num_epochs * 0.1)

finetuned_model.fit(train_objectives=[(train_dataloader, train_loss)], 
                    epochs=num_epochs, 
                    warmup_steps=warmup_steps,
                    output_path="saved_models/rubert-tiny2-wikiutmn")


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/45 [00:00<?, ?it/s]

Iteration:   0%|          | 0/45 [00:00<?, ?it/s]

Iteration:   0%|          | 0/45 [00:00<?, ?it/s]

Iteration:   0%|          | 0/45 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/45 [00:00<?, ?it/s]

Iteration:   0%|          | 0/45 [00:00<?, ?it/s]

Iteration:   0%|          | 0/45 [00:00<?, ?it/s]

Iteration:   0%|          | 0/45 [00:00<?, ?it/s]

Iteration:   0%|          | 0/45 [00:00<?, ?it/s]

### Индексация

In [9]:
finetuned_model = SentenceTransformer('saved_models/rubert-tiny2-wikiutmn', device="cpu")
finetuned_model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 2048, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 312, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
  (2): Normalize()
)

In [15]:
from sqlalchemy import select

with Session(engine) as session:
   documents = session.scalars(select(Chunk).order_by(Chunk.id)).all()
   for doc in documents:
      doc.rusbert_finetuned = finetuned_model.encode(doc.text)
      session.add(doc)
      session.flush()
   session.commit()

In [16]:
def answer_rusbert_finetuned(question):
    with Session(engine) as session:
        return session.scalars(select(Chunk)
                        .order_by(Chunk.rusbert_finetuned.cosine_distance(
                            finetuned_model.encode(question)
                            )).limit(1)).first().text

In [17]:
answer_rusbert_finetuned("Как поменять физкультуру?")

'Выбор спортивных секций по Физической культуре будет проходить в ИС Модеус во вкладке " Выбор модулей ". Вам нужно будет выбрать 2 интересующие Вас спортивные секции, которые будут проходить каждую неделю в одно и то же время. Ограничения : 1 ) Записаться можно не более чем на 2 занятия в неделю 2 ) Нельзя записываться на два занятия подряд. Вас могут не допустить на занятие, если Вы были на предыдущей паре и / или уже посетили два занятия за неделю. При этом расписание на наличие конфликтов Вы проверяете самостоятельно в соответствии с Вашим расписанием в ИС Модеус и расписанием спортивных секций ( во вложенных файлах ). Ваш выбор пролонгируется до конца семестра, однако в любой момент Вы можете его изменить, отписавшись от одной секции и записавшись на другую. ВАЖНО! Студент, пропустивший два занятия подряд, будет отписан автоматически. Выбор Физической культуры откроется 06. 09. 2023 и будет открыт до конца семестра. Для успешной аттестации по дисциплине « Физическая культура : эле

### Метрики

In [18]:
test_questions["rusbert_finetuned"] = test_questions["question"].apply(answer_rusbert_finetuned)
test_questions

,question,document,rusbert_finetuned
0,Я потерял магнитную карту. К кому обратиться?,Для восстановления студенческого билета Вам не...,Для восстановления студенческого билета Вам не...
1,Даты подачи заявления для восстановления на очку,Заявления на восстановление в Университет по о...,Пакет документов для восстановления вам необхо...
2,Что делать при потере проходки?,Для восстановления студенческого билета Вам не...,Для восстановления студенческого билета Вам не...
3,Я хожу в фитнес-клуб. Как заменить физкультуру?,Выбор спортивных секций по Физической культуре...,Выбор спортивных секций по Физической культуре...
4,Когда мне выдадут студенческий билет после пер...,"1. Подать заявление о переводе можно лично, об...","1. Подать заявление о переводе можно лично, об..."
...,...,...,...
62,как закрыть физкультуру?,Выбор спортивных секций по Физической культуре...,Выбор спортивных секций по Физической культуре...
63,когда можно поменять элективы?,"Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи..."
64,как получить справку о месте учёбы?,Студенты очной формы обучения оформляют справк...,Студенты очной формы обучения оформляют справк...
65,Как можно получать баллы за физру?,Выбор спортивных секций по Физической культуре...,Выбор спортивных секций по Физической культуре...


#### Accuracy

In [19]:
print("rusbert_finetuned", sum(test_questions["rusbert_finetuned"].apply(lambda x: "" if x is None else x) == test_questions.document) / len(test_questions.document))

rusbert_finetuned 0.7014925373134329


#### ROUGE-L

In [20]:
from rouge import Rouge
rouge = Rouge()

print("rusbert_finetuned", rouge.get_scores(test_questions["rusbert_finetuned"].apply(lambda x: "-" if x is None else x), test_questions["document"], avg=True)['rouge-l'])

rusbert_finetuned {'r': 0.7733489022002412, 'p': 0.7566198123094525, 'f': 0.7534038029522858}


### Save to Hub

In [21]:
finetuned_model.save_to_hub(repo_id="nizamovtimur/rubert-tiny2-wikiutmn", token=hf_write_token)

model.safetensors:   0%|          | 0.00/117M [00:00<?, ?B/s]

'https://huggingface.co/nizamovtimur/rubert-tiny2-wikiutmn/commit/740236cd3cf4e8acb90ebc9012ca6afcbf07da48'

#### Датасет

In [ ]:
%pip install datasets --quiet

In [8]:
from datasets import DatasetDict, Dataset
dataset_to_hf = DatasetDict({
    'train': Dataset.from_pandas(train_questions),
    'test': Dataset.from_pandas(test_questions)
})
dataset_to_hf.push_to_hub(repo_id="nizamovtimur/wikiutmn-study-gigachat", token=hf_write_token)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

d:\git\virtassist\.venv\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Timur\.cache\huggingface\hub\datasets--nizamovtimur--wikiutmn-study-gigachat. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/datasets/nizamovtimur/wikiutmn-study-gigachat/commit/05b3364a2f11ed902363253c7a9c8537cb1df827', commit_message='Upload dataset', commit_description='', oid='05b3364a2f11ed902363253c7a9c8537cb1df827', pr_url=None, pr_revision=None, pr_num=None)

## Подбор порога "бреда"

TODO: эмпирически подобрать минимальное косинусное расстояние

## Большие языковые модели

https://python.langchain.com/docs/use_cases/question_answering/

In [6]:
from langchain.prompts import PromptTemplate

prompt_template = """
Используй следующий текст в тройных кавычках, чтобы кратко ответить на вопрос студента в конце. 
Не изменяй и не убирай ссылки, адреса и телефоны. Если ты не можешь найти ответ, напиши, что ответ не найден.
Ответ не должен превышать 100 слов.

\"\"\"
{context}
\"\"\"

Вопрос: {question}
"""

prompt = PromptTemplate.from_template(prompt_template)

### GigaChat

In [7]:
from langchain.llms import GigaChat
giga = GigaChat(credentials=gigachat_token, verify_ssl_certs=False)
giga_chain = prompt | giga

In [14]:
# question = questions.question[89]
question = "Я хожу в фитнес-клуб. Как заменить физкультуру?"
print(question, end="\n\n")
document = answer_rusbert_finetuned(question)
print(document)

Я хожу в фитнес-клуб. Как заменить физкультуру?

Выбор спортивных секций по Физической культуре будет проходить в ИС Модеус во вкладке "Выбор модулей".  Вам нужно будет выбрать 2 интересующие Вас спортивные секции, которые будут проходить каждую неделю в одно и то же время.  Ограничения: 1) Записаться можно не более чем на 2 занятия в неделю 2) Нельзя записываться на два занятия подряд.  Вас могут не допустить на занятие, если Вы были на предыдущей паре и/или уже посетили два занятия за неделю. При этом расписание на наличие конфликтов Вы проверяете самостоятельно в соответствии с Вашим расписанием в ИС Модеус и расписанием спортивных секций (во вложенных файлах). Ваш выбор пролонгируется до конца семестра, однако в любой момент Вы можете его изменить, отписавшись от одной секции и записавшись на другую. ВАЖНО!  Студент, пропустивший два занятия подряд, будет отписан автоматически. Выбор Физической культуры откроется 06.09.2023 и будет открыт до конца семестра.  Для успешной аттестации

In [15]:
query = {"context": document,
        "question": question}
print(giga_chain.invoke(query).strip())
print()
print(query)

Для замены физкультуры на посещение спортивного зала необходимо представить тренеру-преподавателю по спорту (куратору по физкультуре) пакет документов, указанных в Положении о реализации дисциплин по физической культуре и спорту. Все документы должны быть предоставлены в течение первой учебной недели.

{'context': 'Выбор спортивных секций по Физической культуре будет проходить в ИС Модеус во вкладке "Выбор модулей".\xa0 Вам нужно будет выбрать 2 интересующие Вас спортивные секции, которые будут проходить каждую неделю в одно и то же время.\xa0 Ограничения: 1) Записаться можно не более чем на 2 занятия в неделю 2) Нельзя записываться на два занятия подряд.\xa0 Вас могут не допустить на занятие, если Вы были на предыдущей паре и/или уже посетили два занятия за неделю. При этом расписание на наличие конфликтов Вы проверяете самостоятельно в соответствии с Вашим расписанием в ИС Модеус и расписанием спортивных секций (во вложенных файлах). Ваш выбор пролонгируется до конца семестра, однако

### Локальные Text2Text

In [16]:
import torch
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

device = torch.cuda.current_device() if torch.cuda.is_available() and torch.cuda.mem_get_info()[0] >= 2*1024**3 else -1
device

0

TODO: `ai-forever/rugpt3large_based_on_gpt2`

In [26]:
# FOR SAVING MODEL ONLY!
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# model_name = "IlyaGusev/fred_t5_ru_turbo_alpaca"
# model_name = "ai-forever/FRED-T5-large"
# model_name = "ai-forever/FRED-T5-1.7B"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
# pipe = pipeline(
#     "text2text-generation", model=model, tokenizer=tokenizer, max_new_tokens=500
# )
# pipe.save_pretrained("saved_models/FRED-T5-1.7B")

tokenizer_config.json: 100%|██████████| 20.2k/20.2k [00:00<00:00, 15.1MB/s]
d:\git\virtassist\.venv\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Timur\.cache\huggingface\hub\models--ai-forever--FRED-T5-1.7B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
vocab.json: 100%|██████████| 1.71M/1.71M 

In [17]:
saved_pipeline = pipeline("text2text-generation", "saved_models/FRED-T5-1.7B", device=device, max_new_tokens=10000)
hf_model = HuggingFacePipeline(pipeline=saved_pipeline).bind(stop=["\n\n"])
gpu_chain = prompt | hf_model

Loading checkpoint shards: 100%|██████████| 2/2 [01:35<00:00, 47.53s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [19]:
query = {"context": document,
        "question": question}
print(gpu_chain.invoke(query).strip())
print()
print(query)

Ответ:
В соответствии с приказом ректора № 555 от 20.08.2023 «О реализации дополнительных образовательных программ в рамках реализации ФГОС ВО» в университете реализуются дополнительные образовательные программы в рамках реализации ФГОС ВО.  В соответствии с приказом ректора № 555 от 20.08.2023 «О реализации дополнительных образовательных программ в рамках реализации ФГОС ВО» в университете реализуются дополнительные образовательные программы в рамках реализации ФГОС ВО.  В соответствии с приказом ректора № 555 от 20.08.2023 «О реализации дополнительных образовательных программ в рамках реализации ФГОС ВО» в университете реализуются дополнительные образовательные программы в рамках реализации ФГОС ВО.  В соответствии с приказом ректора № 555 от 20.08.2023 «О реализации дополнительных образовательных программ в рамках реализации ФГОС ВО» в университете реализуются дополнительные образовательные программы в рамках реализации ФГОС ВО.  В соответствии с приказом ректора № 555 от 20.08.2023

#### Гигиена

In [20]:
del gpu_chain
del hf_model
del saved_pipeline
torch.cuda.empty_cache()